# Spell correction system implementation (NLP course)

# Table of Content
## 1. Introduction
### 1.1 Task
### 1.2 Our approach
### 1.3 Corpus used
### 1.4 Generating and ranking candidates 
### 1.5 Perfomance evaluation
### 1.6 Wrap-up
## 2. Creating the dictionary
## 3. Norviq spell corrector (baseline model)
## 4. Phonetics matching algorithms
### 4.1 Soundex
### 4.2 Metaphone
### 4.3 MRA (match rating approach)
## 5 Word distance matching algorithms
### 5.1 Levenstein distance
### 5.2 Damerau-Levenstien distance
### 5.3 Jaro-Whinkler distance
### 5.4 Hamming distance
### 5.5 Levenstein and Damerau-Levenstien weighted based on Keyboard Physical Distance (QWERTY)
## 6 Ngrams (based on characters in the words)
### 5.1 Unigrams
### 5.2 Bigrams
## 7 Models comparison
## 8 User interface for system testing

# Introduction

## Task

Build a spell correction system (it's quite open ended)

## Our approach

Spell checking is an essential part of many applications. However, its working characteristics (speed, quality, memory consumption) are often not optimal.

10+ years ago Peter Norvig has put his sterlingly simple spell-checker into 21 line of python code. 

https://norvig.com/spell-correct.html

A pure probability model selects the most probable replacement for an unknown word from the list of words collected in a book – this minimalistic approach is still quite relevant and can be considered as a baseline and a prototype for further development. 

However, it has a number of flaws:

 - If our dictionary will be essentially big (for instance, like millions of words), the candidate list will be too big to rank it properly with this simple technique
 - No restrictions on the speed and memory are provided for this solution
 - It cannot find real-word errors

As the starting point (baseline model) we will take Peter Norvig spell corrector and try to compare/beat it with the algorithms we have learned during the NLP course:
 - Phonetics group 
 - Word distance matching techniques 
 - Ngrams (based on characters in the words)

## Corpus 

We are using slightly pre-processed 'big.txt' file, that can be obtained from Peter Norvig’s web site.

https://www.norvig.com/ngrams/

The corpus contains a concatenation of several public domain books from Project Gutenberg and lists of most frequent words from Wiktionary and the British National Corpus. We can assume that the words in the 'big.txt' file are spelled correctly. 

In order to validate our model we will use the a list of 384 misspelled words (test-words-misspelled.txt) and their corresponding correct spellings (test-words-correct.txt)

## Generating and ranking candidates 


For each algorithm the generation of candidates was implemented differently:

 - By choosing the strings' minimum betweeness distance (if the algorithm is measuring the actual distance between two strings: Phonetics and most of Edit Distance models)
 - By choosing the words with maximum index distance (if the algorithm is measuring the distance in the index format: from 0 to 1, Jaro-Winkler and Levenstien Wieghted Keyboard distance)
 - Ngrams (by n-characters frequency distribution comparison)

 
Ranking of final pool of candidates was implemented in accordance with probablity of candidate occurence in a given corpus (as done for Norviq's version)

## Evaluation Metrics

We have evaluted algorithms based on two performance aspects:

 - Accuracy: Algorithms predict only one most possible candidate (headword). When it is exact the correct headword, we count that as a correct prediction. Then calculate the proportion of the number of being successfully predicted to the number of attempted headwords. High accuracy means the method can output a single word and the word is the right prediction.

 - Time cost: Time will be recorded from the start of the program to the end. And this includes the time spent on calculating accuracy.


For the future development it is recommended to take into account the following two metrics as well:

 - Precision (Algorithms predict one or more possible candidates. Count as a correct prediction when the correct candidate is in   the result. Calculate the proportion of the number of being successfully predicted to the number of whole prediction result). High precision means the method can output the right word out of not too many predictions

 - Recall ratio (Algorithms predict one or more possible candidates. Based on that we calculate the proportion of the number of being successfully predicted to the number of attempted variants)

## Wrap-up

### The results
The results showed the best perfomance for Damerau-Levenstein approach, which has slightly outperfomed Norviq's accuracy (73.96 % vs. 73.7 %). But underscores on time speed (29.3 vs 8.4 seconds for correction)


### Possible Additional Improvements

During the system testing we have explored that searching of a word in a plain dictionary or list structure can take a while (especially for some computational consuming algorithms), and it can take even more if you should measure a distance between an out-of-vocabulary word and every word in a dictionary to find corrections.

That's why for the further time cost optimizations it will be advantageous to switch to an ordered tree data structure used to store a dynamic set or associative array where the keys are usually strings. It is kind of similar to a binary search tree, but for language data - you have no > < conditions

https://en.wikipedia.org/wiki/Trie

The other option could be the usage of bk-trees. Instead of plain dictionaries is that measuring the distance between an out-of-vocabulary word and every word in the dictionary is much faster in this kind of structure - it’s now O(log n) instead of O(n).

https://en.wikipedia.org/wiki/BK-tree

# Creating the dictionary 

In [137]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline
import re
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math
from collections import Counter
import time
import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [138]:
# loading the text
text = open('C:/Users/serge/Desktop/Datasets/big.txt', 'r', encoding='utf-8').read()

def clean_text(text):
    # Remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    
    # Remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    
    # Substitute multiple whitespace with single whitespace # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

cleaned_text = clean_text(text)


def tokenize (raw):
   
    #Tokenizening
    words = word_tokenize(raw)
    
    #Filtering the words (since we have a test sample with words length>2)
    words=[words for words in words if len(words) > 2]

    # Deleting stop_words
    stop_words = stopwords.words('English')
    words = [i for i in words if (i not in stop_words)]
    

    return words

vocab=tokenize (cleaned_text) 

In [139]:
#Total number of words in dict:
len(vocab)

547217

In [140]:
#Total number of unique words
vocabulary=set(vocab)
len(vocabulary)

35168

In [141]:
vocabulary=list(vocabulary)
vocabulary[:30]

['expulsion',
 'packed',
 'berkshire',
 'filter',
 'choirs',
 'spongiopilene',
 'combinations',
 'carefree',
 'threaten',
 'gibe',
 'hydraulics',
 'diaz',
 'powder',
 'hobart',
 'dont',
 'fragments',
 'byproducts',
 'classes',
 'bosnia',
 'phleboliths',
 'faculty',
 'immensity',
 'prepatellar',
 'reflections',
 'conversion',
 'unsolicited',
 'exasperated',
 'peasantsstreamed',
 'danielll',
 'explanatory']

## Norvig Spell corrector as the starting point

Norvig spell corrector finds out all the words that ate one or two or multiple edit distances away from the given word. It finally assigns correct spelling to the word which has the maximum probability of occurence in a given corpus.

In [142]:
WORDS = Counter(vocab)

wordcount = []
for value in WORDS.values():
    wordcount.append(value)

total = sum(wordcount)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def P(word): 
    N = total
    "Probability of `word`."
    return (WORDS[word] / N)

In [143]:
#Just some EDA to explore the file we use as bag of words
WORDS.most_common(30)

[('said', 3456),
 ('one', 3215),
 ('may', 2538),
 ('would', 1949),
 ('prince', 1893),
 ('pierre', 1785),
 ('could', 1695),
 ('time', 1509),
 ('man', 1502),
 ('new', 1200),
 ('first', 1155),
 ('well', 1143),
 ('old', 1138),
 ('face', 1122),
 ('upon', 1108),
 ('men', 1104),
 ('see', 1094),
 ('natasha', 1093),
 ('two', 1071),
 ('andrew', 1065),
 ('french', 1059),
 ('know', 1041),
 ('like', 1017),
 ('without', 1006),
 ('went', 1005),
 ('made', 999),
 ('little', 997),
 ('came', 976),
 ('states', 963),
 ('must', 954)]

In [144]:
#Most popular word
max(WORDS, key=P)

'said'

In [145]:
#Printing the candidates for correction
candidates('treas')

{'areas', 'tread', 'treat', 'treats', 'trees', 'tres'}

In [146]:
#Printing the actual correction
correction('treas')

'trees'

In [147]:
list_cand=list(candidates('treas'))

In [148]:
#We can see that it ranks candidates according to probability in the dictionary
for i in list_cand:
    print (i, P(i))

tread 2.1929143283194784e-05
tres 7.309714427731595e-06
treat 5.665028681491986e-05
areas 7.492457288424885e-05
trees 9.319885895357783e-05
treats 3.6548572138657973e-06


# Phonetic Matching Algorithms

Norvig Spell corrector is using standard distance measure, which is based on letter alignment.
That's why this search is limited to candidates standing 1-2 letters from a word with an error.

In [149]:
# Let's test new things
correction('treeeees')

'treeless'

In [150]:
correction('riiiiight')

'riiiiight'

In [151]:
correction('riiight')

'right'

We can observe that Norvig Spell corrector can't handle such type of mistakes, because it is a typical errors, caused by intentional distortion or slangy language gamification, outstand from a relevant candidate more than 2 letters away

In order to test how Phonetics algorithms will deal with this we will use pyphonetics library and test three models: Soundex (its refined version), Metaphone and Match rating approach. In order to calculate rank candidates we will use method 'distance'  to find the distance between 2 phonetic representations

## Soundex

In [152]:
#!pip install pyphonetics

In [153]:
from pyphonetics import Soundex
from pyphonetics import RefinedSoundex

Soundex is a phonetic algorithm for indexing names by sound, as pronounced in English. The goal is for homophones to be encoded to the same representation so that they can be matched despite minor differences in spelling.The algorithm mainly encodes consonants; a vowel will not be encoded unless it is the first letter. Method was originally developed by Margaret Odell and Robert Russell. 

https://en.wikipedia.org/wiki/Soundex

In [154]:
soundex = RefinedSoundex()
soundex.phonetics('treeeeees')

'T6903'

In [155]:
soundex.phonetics('trees')

'T6903'

In [156]:
soundex.distance('treeeeees', 'trees')

0

In [157]:
def candidates_soundex(word):
    soundex = RefinedSoundex()
    cand_list =[]
    best_soundex= 0
    for current_string in vocabulary:
        current_score = soundex.distance(word, current_string)
        if(current_score == best_soundex):
            best_soundex = current_score
            cand_list.append(current_string)
    return cand_list

In [158]:
candidates_soundex('treeeeees')

['truck',
 'truss',
 'tres',
 'threes',
 'trash',
 'tricks',
 'thresh',
 'trees',
 'trick',
 'throws',
 'tracks',
 'tries',
 'track']

In [159]:
#Let's try more complex one with riiiiight
candidates_soundex('riiiiight')

['right']

In [160]:
def correction_soundex_bestprobability(word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    try:
        return max(candidates_soundex (word), key=P)
    except:
        ValueError

In [161]:
correction_soundex_bestprobability ('treeeeees')

'trees'

In [162]:
correction_soundex_bestprobability('riiiiight')

'right'

The results of correction are impressive

## Metaphone

In [163]:
from pyphonetics import Metaphone

Metaphone is a phonetic algorithm, published by Lawrence Philips in 1990, for indexing words by their English pronunciation. It fundamentally improves on the Soundex algorithm by using information about variations and inconsistencies in English spelling and pronunciation to produce a more accurate encoding, which does a better job of matching words and names which sound similar. As with Soundex, similar-sounding words should share the same keys.

https://en.wikipedia.org/wiki/Metaphone

In more simple words Metaphone makes a phonetic hash out of each word in the vocabulary and getting this kind of a hash for a word with error, you can search for a better candidate through your hashed dictionary using standard distance measures.

In [164]:
metaphone=Metaphone()
metaphone.phonetics('treeeeees')

'TRS'

In [165]:
metaphone.phonetics('trees')

'TRS'

In [166]:
metaphone = Metaphone()
metaphone.distance('treeeeees', 'trees')

0

In [167]:
def candidates_metaphone(word):
    metaphone = Metaphone()
    cand_list =[]
    best_metaphone= 0
    for current_string in vocabulary:
        current_score = metaphone.distance(word, current_string)
        if(current_score == best_metaphone):
            best_metaphone = current_score
            cand_list.append(current_string)
    return cand_list

In [168]:
candidates_metaphone('treeeees')

['trace',
 'drowsy',
 'troughs',
 'drissa',
 'diaries',
 'truce',
 'dorsi',
 'truss',
 'tres',
 'trousseau',
 'doers',
 'terse',
 'dorrs',
 'daresay',
 'draws',
 'dares',
 'dries',
 'tierce',
 'trees',
 'dress',
 'tiers',
 'doors',
 'tories',
 'tears',
 'tries',
 'taras',
 'terrace',
 'darcy',
 'tires',
 'teres']

In [169]:
#Let's try more complex one with riiiiight
candidates_metaphone('riiiiight')

['ride',
 'riot',
 'ready',
 'road',
 'reid',
 'wrought',
 'rid',
 'read',
 'rude',
 'rod',
 'ruddy',
 'root',
 'reed',
 'raid',
 'rowdy',
 'rat',
 'rout',
 'red',
 'rite',
 'write',
 'wright',
 'rode',
 'rut',
 'wrote',
 'writ',
 'right',
 'rot',
 'route',
 'rate',
 'rete',
 'radio']

In [170]:
def correction_metaphone_bestprobability(word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    try:
        return max(candidates_metaphone (word), key=P)
    except:
        ValueError

In [171]:
correction_metaphone_bestprobability ('treeeees')

'tears'

In [172]:
correction_metaphone_bestprobability ('riiiiight')

'right'

## Match Rating Approach

The match rating approach (MRA) is a phonetic algorithm developed by Western Airlines in 1977 for the indexation and comparison of homophonous names.

The algorithm itself has a simple set of encoding rules but a more lengthy set of comparison rules. The main mechanism is the similarity comparison, which calculates the number of unmatched characters by comparing the strings from left to right and then from right to left, and removing identical characters. This value is subtracted from 6 and then compared to a minimum threshold. The minimum threshold is defined in table A and is dependent upon the length of the strings.

The encoded name is known (perhaps incorrectly) as a personal numeric identifier (PNI). The encoded name can never contain more than 6 alpha only characters.


https://en.wikipedia.org/wiki/Match_rating_approach

Unlike Metaphone, MRA includes both hashing rules and their distance measure. It is suitable for small vocabularies and searching for abbreviations and acronyms.

In [173]:
from pyphonetics import MatchingRatingApproach

In [174]:
def candidates_mra(word):
    mra= MatchingRatingApproach()
    cand_list =[]
    best_mra= 0
    for current_string in vocabulary:
        current_score = mra.distance(word, current_string)
        if(current_score == best_mra):
            best_mra = current_score
            cand_list.append(current_string)
    return cand_list

In [175]:
candidates_mra('treeees')

['tresses',
 'truss',
 'tres',
 'tutors',
 'trousseau',
 'tarsus',
 'terse',
 'terrors',
 'trees',
 'tiers',
 'tories',
 'tears',
 'tries',
 'taras',
 'tires',
 'teres']

In [176]:
#Let's try more complex one with riiiiight
candidates_mra('riiiiight')

['right']

In [177]:
def correction_mra_bestprobability(word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    try:
        return max(candidates_mra (word), key=P)
    except:
        ValueError

In [178]:
correction_mra_bestprobability('treeees')

'tears'

In [179]:
#Let's try more complex one with riiiight
correction_mra_bestprobability('riiiiight')

'right'

So far we can see that RefinedSoundex is outperfoming Metaphone and MRA on two examples. But we will come back to more robust testing later

# Word distance algortihms

Distance measures are methods that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other.

In this part we will compare different types of edit distance models:
- Levenshtein Distance
- Damerau-Levenshtein Distance
- Jaro-Winkler Distance
- Hamming Distance

In order to play with different distance algorithms we will rely on python jellyfish library and impelement our customized functions to asess and choose the candidates

Using the above edit distance, we will write a function that calculates all words with minimal edit distance to the misspelled word. 
The sub tasks involved are: 
- Collect the set of all unique words in vocabulary
- Find the minimal edit distance, that is the lowest value for the function edit_distance between token and a word in train
- Output all unique words in train that have this same (minimal) edit_distance value

In [180]:
#!pip install jellyfish
import jellyfish

## Levenshtein Distance

The Levenshtein distance is a string metric for measuring the difference between two sequences. Informally, the Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other. It is named after the Soviet mathematician Vladimir Levenshtein, who considered this distance in 1965

https://en.wikipedia.org/wiki/Levenshtein_distance

Allows deletion, insertion and substitution

In [181]:
# Edit distance returns the number of changes to transform one word to another
jellyfish.levenshtein_distance('simone', 'siomne')

2

In [182]:
def candidates_lev (word):
    #Creating List containing edit distances
    edit_list=[]
    
    #Creating list containg possible candidates for right spelling
    cand_list=[]
    
    for i in range(0,len(vocabulary)):
        edit_list.append(jellyfish.levenshtein_distance(word,vocabulary[i]))
        
    #Getting the minimum value for edit distance
    minimun=min(edit_list)
    
    for i in range(0,len(vocabulary)):
        if(jellyfish.levenshtein_distance(word, vocabulary[i])==minimun):
            cand_list.append(vocabulary[i])
    return cand_list

In [183]:
candidates_lev('siomne')

['stone', 'shone', 'omne', 'sloane']

In [184]:
def correction_lev(word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    return max(candidates_lev(word), key=P)

In [185]:
correction_lev('siomne')

'stone'

## Damerau-Levenshtein Distance

The Damerau–Levenshtein distance (named after Frederick J. Damerau and Vladimir I. Levenshtein is a string metric for measuring the edit distance between two sequences. Informally, the Damerau–Levenshtein distance between two words is the minimum number of operations (consisting of insertions, deletions or substitutions of a single character, or transposition of two adjacent characters) required to change one word into the other.


https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance

As compared to Levenstein one it takes into account the transposition of two adjacent characters

In [186]:
jellyfish.damerau_levenshtein_distance('simone', 'siomne')

1

In [187]:
def candidates_dam_lev (word):
    #Creating List containing edit distances
    edit_list=[]
    
    #Creating list containg possible candidates for right spelling
    cand_list=[]
    
    for i in range(0,len(vocabulary)):
        edit_list.append(jellyfish.damerau_levenshtein_distance(word,vocabulary[i]))
        
    #Getting the minimum value for edit distance
    minimun=min(edit_list)
    
    for i in range(0,len(vocabulary)):
        if(jellyfish.damerau_levenshtein_distance(word, vocabulary[i])==minimun):
            cand_list.append(vocabulary[i])
    
    return cand_list

In [188]:
candidates_dam_lev('simone')

['simons', 'simon']

In [189]:
def correction_dam_lev(word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    return max(candidates_dam_lev(word), key=P)

In [190]:
correction_dam_lev('simone')

'simon'

## Jaro-Winkler Distance

Measures the minimum number of single-character transpositions required to change one word into the other, giving more favorable ratings to strings that match from the beginning

https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance

In [191]:
jellyfish.jaro_winkler('simone', 'siomne')

0.9555555555555556

In [192]:
jellyfish.jaro_winkler('treeeeeeeeeees', 'trees')

0.6952380952380953

In [193]:
jellyfish.jaro_winkler('no', 'no')

1.0

In [194]:
def candidates_jaro_winkler(word):
    #Creating List containing edit distances
    edit_list=[]
    
    #Creating list containg possible candidates for right spelling
    cand_list=[]
    
    for i in range(0,len(vocabulary)):
        edit_list.append(jellyfish.jaro_winkler(word,vocabulary[i]))
        
    #Getting the maximum value for edit distance
    maximum=max(edit_list)
    
    for i in range(0,len(vocabulary)):
        if(jellyfish.jaro_winkler(word, vocabulary[i])==maximum):
            cand_list.append(vocabulary[i])
    
    return cand_list

In [195]:
candidates_jaro_winkler('siomne')

['simon']

In [196]:
def correction_jaro_winkler(word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    return max(candidates_jaro_winkler(word), key=P)

In [197]:
correction_jaro_winkler('siomne')

'simon'

## Hamming distance

The Hamming distance between two strings of equal length is the number of positions at which the corresponding symbols are different. In other words, it measures the minimum number of substitutions required to change one string into the other, or the minimum number of errors that could have transformed one string into the other. In a more general context, the Hamming distance is one of several string metrics for measuring the edit distance between two sequences. It is named after the American mathematician Richard Hamming.

https://en.wikipedia.org/wiki/Hamming_distance

Allows only substitution, hence, it only applies to strings of the same length

In [198]:
jellyfish.hamming_distance('treek', 'trees')

1

In [199]:
def candidates_hamm (word):
    #Creating List containing edit distances
    edit_list=[]
    
    #Creating list containg possible candidates for right spelling
    cand_list=[]
    
    for i in range(0,len(vocabulary)):
        edit_list.append(jellyfish.hamming_distance(word,vocab[i]))
        
    #Getting the minimum value for edit distance
    minimun=min(edit_list)
    
    for i in range(0,len(vocabulary)):
        if(jellyfish.hamming_distance(word, vocabulary[i])==minimun):
            cand_list.append(vocabulary[i])
    
    return cand_list

In [200]:
candidates_hamm ('treek')

['creek', 'greek', 'tree', 'trees']

In [201]:
def correction_hamm (word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    return max(candidates_hamm (word), key=P)

In [202]:
correction_hamm ('treek')

'trees'

## Leventsein and Damerau-Levenshtein weighted on keyboards distance

In this part we want to compare two strings, and and allow for typos, based on the physical distance between keys on Qwerty keyboard. 

In other words, the w algorithm should prefer "yelephone" to "zelephone" since the "y" key is located nearer to the "t" key than to the "z" key on most keyboards

In [203]:
from math import sqrt

keyboard_cartesian = {'q': {'x':0, 'y':0}, 'w': {'x':1, 'y':0}, 'e': {'x':2, 'y':0}, 'r': {'x':3, 'y':0}, 't': {'x':4, 'y':0}, 'y': {'x':5, 'y':0}, 'u': {'x':6, 'y':0}, 'i': {'x':7, 'y':0}, 'o': {'x':8, 'y':0}, 'p': {'x':9, 'y':0}, 'a': {'x':0, 'y':1},'z': {'x':0, 'y':2},'s': {'x':1, 'y':1},'x': {'x':1, 'y':2},'d': {'x':2, 'y':1},'c': {'x':2, 'y':2}, 'f': {'x':3, 'y':1}, 'b': {'x':4, 'y':2}, 'm': {'x':5, 'y':2}, 'j': {'x':6, 'y':1}, 'g': {'x':4, 'y':1}, 'h': {'x':5, 'y':1}, 'j': {'x':6, 'y':1}, 'k': {'x':7, 'y':1}, 'l': {'x':8, 'y':1}, 'v': {'x':3, 'y':2}, 'n': {'x':5, 'y':2}, }

def keyboard_distance(a,b):
    X = (keyboard_cartesian[a]['x'] - keyboard_cartesian[b]['x'])**2
    Y = (keyboard_cartesian[a]['y'] - keyboard_cartesian[b]['y'])**2
    return math.sqrt(X+Y) 

In [204]:
#Testing closest one
keyboard_distance ('q','w')

1.0

In [205]:
#Testing the faraway  one
keyboard_distance('z','p')

9.219544457292887

Taking into account this development, we created the own probability function, weighted by qwerty keyboard distance: 

https://metacpan.org/pod/release/KRBURTON/String-KeyboardDistance-1.01/KeyboardDistance.pm#qwerty_keyboard_distance_match


Pr = 1 - ( D / (L * M) )
Where D is the distance between the two strings, L is the length of the longer string, and M is the maximum character distance.

In [206]:
import jellyfish

def weighted_lev_distance (string1, string2):
    
    key_dist_lev = []
    
    max_len=max(len(string1), len(string2))

      
    def splitter1(string1):
        charlist = list(string1) 
        letters1 = ' '.join(charlist).split()
        return letters1

    
    def splitter2(string2):
        charlist = list(string2) 
        letters2= ' '.join(charlist).split()
        return letters2
    
    for a in splitter1(string1):
        for b in splitter2(string2):
            key_dist_lev.append(keyboard_distance(a,b))
    max_key_dist_lev=(max(key_dist_lev))   
     
    proba= (1 - (jellyfish.levenshtein_distance(string1, string2)/(max_len*max_key_dist_lev)))
    return proba

In [207]:
def weighted_dau_lev_distance (string1, string2):
    
    key_dist_dau_lev = []
    
    max_len=max(len(string1), len(string2))

      
    def splitter1(string1):
        charlist = list(string1) 
        letters1 = ' '.join(charlist).split()
        return letters1

    
    def splitter2(string2):
        charlist = list(string2) 
        letters2= ' '.join(charlist).split()
        return letters2
    
    for a in splitter1(string1):
        for b in splitter2(string2):
            key_dist_dau_lev.append(keyboard_distance(a,b))
    max_key_dist_dau_lev=(max(key_dist_dau_lev))   
     
    proba= (1 - (jellyfish.damerau_levenshtein_distance(string1, string2)/(max_len*max_key_dist_dau_lev)))
    return proba

In [208]:
weighted_dau_lev_distance ('no', 'no')

1.0

In [209]:
weighted_dau_lev_distance ('no', 'onoooooooo')

0.7781199215099084

In [210]:
weighted_dau_lev_distance ('telephone', 'yelephone')

0.9841269841269842

In [211]:
weighted_dau_lev_distance ('telephone', 'oelephone')

0.9841269841269842

In [212]:
def candidates_dam_lev_keyboard (word):
    #Creating List containing edit distances
    probability_list=[]
    
    #Creating list containg possible candidates for right spelling
    cand_list=[]
    
    for i in range(0,len(vocabulary)):
        probability_list.append(weighted_dau_lev_distance(word,vocabulary[i]))
        
    #Getting the max probabilit for edit distance
    maximum=max(probability_list)
    
    for i in range(0,len(vocabulary)):
        if(weighted_dau_lev_distance(word, vocabulary[i])==maximum):
            cand_list.append(vocabulary[i])
    
    return cand_list

In [213]:
candidates_dam_lev_keyboard ('zzyelephone')

['telephone']

In [214]:
def correction_dam_lev_keyboard(word): 
    #There is no specifications how to choose the candidate, we will use the most probable spelling correction for word
    return max(candidates_dam_lev_keyboard(word), key=P)

In [215]:
correction_dam_lev_keyboard ('zzyelephone')

'telephone'

Briefly comparing results with rest methods 

In [216]:
correction('zzyelephone')

'zzyelephone'

In [217]:
correction_lev('zzyelephone')

'telephone'

In [218]:
correction_dam_lev('zzyelephone')

'telephone'

# Ngrams

In general, n-gram means splitting a string in sequences with the length n. So if we have this string “abcde”, then bigrams are: ab, bc, cd, and de while trigrams will be: abc, bcd, and cde while 4-grams will be abcd, and bcde.

Since out testing words consist of 3 words+ we will test only the perfomance of unigrams use bigrams

In [219]:
from nltk.util import ngrams, bigrams

In [220]:
string='treeeeees'

## Unigrams 

In [221]:
def correction_unigrams (entries=[string]):
    corrected = []
    dist_old = 1
    best_match = None
    for w in entries:
        w_0=w[0]
        for c in vocabulary:
            dist_new = nltk.jaccard_distance(set(nltk.ngrams(c, n=1)),set(nltk.ngrams(w, n=1)))
            if dist_new<dist_old and w_0==c[0]:
                best_match = c
                dist_old = dist_new 
        corrected.append(best_match)
        return ', '.join(map(str, corrected))
    
correction_unigrams()

'tresses'

## Bigrams 

In [222]:
def correction_bigrams (entries=[string]):
    corrected = []
    dist_old = 1
    best_match = None
    for w in entries:
        w_0=w[0]
        for c in vocabulary:
            dist_new = nltk.jaccard_distance(set(nltk.bigrams(c)),set(nltk.bigrams(w)))
            if dist_new < dist_old and w_0==c[0]:
                best_match = c
                dist_old = dist_new
        corrected.append(best_match)
        corrected.append(best_match)
        return ', '.join(map(str, corrected))
    
correction_bigrams()

'trees, trees'

# Model comparison (evaluation of corrections) 

In [223]:
misspellings = pd.read_excel('C:/Users/serge/Desktop/Datasets/misspelings_short.xlsx')
misp_list = list(misspellings['errors'])
correct_answers = list(misspellings['correct'])

misspellings.shape

(384, 2)

In [224]:
#Creating the initial dataframe

#Misspellings = misspelt words
#Correct_answers = answers of truth set

data = pd.DataFrame({'misspellings':misp_list,'correct_answers':correct_answers})

In [225]:
data.head()

,misspellings,correct_answers
0,abilty,ability
1,abraod,abroad
2,acedemic,academic
3,accesion,accession
4,accomodate,accommodate


## Step-by-step adding the best candidates from all the techniques 

In [226]:
#Norvig (baseline corrector)
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Norviq_speed=(time.time() - start_time)

data['Norviq']=df

--- 8.429903984069824 seconds ---


In [227]:
#Soundex
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_soundex_bestprobability(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Soundex_speed=(time.time() - start_time)

data['Soundex']=df

--- 673.3390271663666 seconds ---


In [228]:
#Metaphone
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_metaphone_bestprobability(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Metaphone_speed=(time.time() - start_time)

data['Metaphone']=df

--- 1868.1607694625854 seconds ---


In [229]:
#MRA
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_mra_bestprobability(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
MRA_speed=(time.time() - start_time)

data['MRA']=df

--- 682.3088150024414 seconds ---


In [230]:
#Leventstein
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_lev(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Lev_speed=(time.time() - start_time)

data['Levenstein']=df

--- 17.26406455039978 seconds ---


In [231]:
#Damerau Levenstein
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_dam_lev(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Damerau_Lev_speed=(time.time() - start_time)

data['Damerau_Levenstein']=df

--- 29.3111891746521 seconds ---


In [232]:
#Jaro Winkler
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_jaro_winkler(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Jaro_Winkler_speed=(time.time() - start_time)

data['Jaro_Winkler']=df

--- 52.08599090576172 seconds ---


In [233]:
#Hamming
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_hamm(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Hamming_speed=(time.time() - start_time)

data['Hamming']=df

--- 11.240901470184326 seconds ---


In [234]:
#Unigrams
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_unigrams(entries=[i]))
    
print("--- %s seconds ---" % (time.time() - start_time))
Unigrams_speed=(time.time() - start_time)

data['Unigrams']=df

--- 128.05072379112244 seconds ---


In [235]:
#Bigrams
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_bigrams(entries=[i]))
    
print("--- %s seconds ---" % (time.time() - start_time))
Bigrams_speed=(time.time() - start_time)

data['Bigrams']=df

--- 152.0656476020813 seconds ---


In [236]:
#Damerau Levenstein Distance weighted by physical Keyboard distance (QWERTY)
start_time = time.time()

df = []
for i in misp_list:
    df.append(correction_dam_lev_keyboard(i))
    
print("--- %s seconds ---" % (time.time() - start_time))
Dam_Lev_Keyboardweighted_speed=(time.time() - start_time)

data['Dam_Lev_Keyboardweighted']=df

--- 3973.498164653778 seconds ---


In [238]:
data.head()

,misspellings,correct_answers,Norviq,Soundex,Metaphone,MRA,Levenstein,Damerau_Levenstein,Jaro_Winkler,Hamming,Unigrams,Bigrams,Dam_Lev_Keyboardweighted
0,abilty,ability,ability,None,ability,ability,ability,ability,ability,guilty,ability,"ability, ability",ability
1,abraod,abroad,abroad,abroad,abroad,abroad,afraid,abroad,abroad,afraid,abroad,"abraham, abraham",abroad
2,acedemic,academic,academic,academic,None,academic,academic,academic,academic,acute,academic,"academic, academic",academic
3,accesion,accession,accession,accession,accession,accession,accession,accession,accession,occasion,accession,"accession, accession",accession
4,accomodate,accommodate,accommodate,accommodate,accommodate,accommodate,accommodate,accommodate,accommodate,accumulated,accommodate,"accommodate, accommodate",accommodate


In [239]:
#Creating binary variable as to see where our function  (CORRECT_TEXT matches the truth set = CORRECT_ANSWERS)
data['Norviq_accuracy']=data.apply(lambda row: row['Norviq'] == row['correct_answers'],axis=1)
data['Soundex_accuracy']=data.apply(lambda row: row['Soundex'] == row['correct_answers'],axis=1)
data['Metaphone_accuracy']=data.apply(lambda row: row['Metaphone'] == row['correct_answers'],axis=1)
data['MRA_accuracy']=data.apply(lambda row: row['MRA'] == row['correct_answers'],axis=1)
data['Levenstein_accuracy']=data.apply(lambda row: row['Levenstein'] == row['correct_answers'],axis=1)
data['Damerau_Levenstein_accuracy']=data.apply(lambda row: row['Damerau_Levenstein'] == row['correct_answers'],axis=1)
data['Jaro_Winkler_accuracy']=data.apply(lambda row: row['Jaro_Winkler'] == row['correct_answers'],axis=1)
data['Hamming_accuracy']=data.apply(lambda row: row['Hamming'] == row['correct_answers'],axis=1)
data['Unigrams_accuracy']=data.apply(lambda row: row['Unigrams'] == row['correct_answers'],axis=1)
data['Bigrams_accuracy']=data.apply(lambda row: row['Bigrams'] == row['correct_answers'],axis=1)
data['Dam_Lev_Keyboardweighted_accuracy']=data.apply(lambda row: row['Dam_Lev_Keyboardweighted'] == row['correct_answers'],axis=1)

In [240]:
data.columns

Index(['misspellings', 'correct_answers', 'Norviq', 'Soundex', 'Metaphone',
       'MRA', 'Levenstein', 'Damerau_Levenstein', 'Jaro_Winkler', 'Hamming',
       'Unigrams', 'Bigrams', 'Dam_Lev_Keyboardweighted', 'Norviq_accuracy',
       'Soundex_accuracy', 'Metaphone_accuracy', 'MRA_accuracy',
       'Levenstein_accuracy', 'Damerau_Levenstein_accuracy',
       'Jaro_Winkler_accuracy', 'Hamming_accuracy', 'Unigrams_accuracy',
       'Bigrams_accuracy', 'Dam_Lev_Keyboardweighted_accuracy'],
      dtype='object')

In [241]:
col_binary=['Norviq_accuracy', 'Soundex_accuracy', 'Metaphone_accuracy', 'MRA_accuracy', 'Levenstein_accuracy',
       'Damerau_Levenstein_accuracy', 'Jaro_Winkler_accuracy',
       'Hamming_accuracy',  'Unigrams_accuracy', 'Bigrams_accuracy', 'Dam_Lev_Keyboardweighted_accuracy']

In [242]:
#This transforms the correct column previously created from TRUE & FALSE to 1 and zero
data[col_binary]=data[col_binary]*1

In [243]:
data.head()

,misspellings,correct_answers,Norviq,Soundex,Metaphone,MRA,Levenstein,Damerau_Levenstein,Jaro_Winkler,Hamming,...,Soundex_accuracy,Metaphone_accuracy,MRA_accuracy,Levenstein_accuracy,Damerau_Levenstein_accuracy,Jaro_Winkler_accuracy,Hamming_accuracy,Unigrams_accuracy,Bigrams_accuracy,Dam_Lev_Keyboardweighted_accuracy
0,abilty,ability,ability,None,ability,ability,ability,ability,ability,guilty,...,0,1,1,1,1,1,0,1,0,1
1,abraod,abroad,abroad,abroad,abroad,abroad,afraid,abroad,abroad,afraid,...,1,1,1,0,1,1,0,1,0,1
2,acedemic,academic,academic,academic,None,academic,academic,academic,academic,acute,...,1,0,1,1,1,1,0,1,0,1
3,accesion,accession,accession,accession,accession,accession,accession,accession,accession,occasion,...,1,1,1,1,1,1,0,1,0,1
4,accomodate,accommodate,accommodate,accommodate,accommodate,accommodate,accommodate,accommodate,accommodate,accumulated,...,1,1,1,1,1,1,0,1,0,1


In [246]:
for col in data[col_binary]:
    display(data[col].value_counts(normalize=True))

1    0.736979
0    0.263021
Name: Norviq_accuracy, dtype: float64

0    0.679688
1    0.320312
Name: Soundex_accuracy, dtype: float64

0    0.601562
1    0.398438
Name: Metaphone_accuracy, dtype: float64

0    0.523438
1    0.476562
Name: MRA_accuracy, dtype: float64

1    0.669271
0    0.330729
Name: Levenstein_accuracy, dtype: float64

...

1    0.700521
0    0.299479
Name: Jaro_Winkler_accuracy, dtype: float64

0    0.802083
1    0.197917
Name: Hamming_accuracy, dtype: float64

1    0.513021
0    0.486979
Name: Unigrams_accuracy, dtype: float64

0    1.0
Name: Bigrams_accuracy, dtype: float64

1    0.721354
0    0.278646
Name: Dam_Lev_Keyboardweighted_accuracy, dtype: float64

In [247]:
speed_measurements= [Norviq_speed, Soundex_speed, Metaphone_speed, MRA_speed, Lev_speed, Damerau_Lev_speed,  Jaro_Winkler_speed, Hamming_speed, Unigrams_speed, Bigrams_speed, Dam_Lev_Keyboardweighted_speed]

In [248]:
for algo in speed_measurements:
    display(np.round(algo,1))

8.4

673.3

1868.2

682.3

17.3

...

52.1

11.2

128.1

152.1

3973.5

In [249]:
statistics=pd.DataFrame(columns={'Accuracy', 'Time speed (in seconds)'}, index=['Norviq', 'Soundex', 'Metaphone', 'MRA', 'Levenstein', 'Damerau-Levenstein', 'Jaro-Winkler', 'Hamming', 'Unigrams', 'Bigrams','Dam-Lev weighted by Keyboard Distance'])

#Calculating the accuracy
accuracy=[]
for col in data[col_binary]:
    accuracy.append(round(sum(data[col]==1)*100/sum(data[col].value_counts()),2))

statistics['Accuracy']=accuracy


#Calculating the speed
speed=[]
for algo in speed_measurements:
    speed.append(np.round(algo,1))
    
statistics['Time speed (in seconds)']=speed

#Showing the results
statistics.sort_values(by='Accuracy', ascending=False)

,Time speed (in seconds),Accuracy
Damerau-Levenstein,29.3,73.96
Norviq,8.4,73.70
Dam-Lev weighted by Keyboard Distance,3973.5,72.14
Jaro-Winkler,52.1,70.05
Levenstein,17.3,66.93
Unigrams,128.1,51.30
MRA,682.3,47.66
Metaphone,1868.2,39.84
Soundex,673.3,32.03
Hamming,11.2,19.79


# User interface

In [269]:
def interface():
    a = input("Enter one word: ")
    b = a 
    print ("Spell-checked version by Group C: ")
    return correction_dam_lev(b)

interface()

Enter one word: sdsd
Spell-checked version by Group C: 


'said'